## 1. Preparation
### 1.1. Import libraries

In [93]:
import folium
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import urllib.request  as urllib2
import pandas as pd
import json
import numpy as np
import requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [74]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### 1.2. Get html and parse it

In [25]:
raw_html = urllib2.urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").read().decode("utf-8") 

# select region of interest - the table with list of postal codes of Toronto
table_start = raw_html.find('<table')
table_end = raw_html.find('</table>')
table_html = raw_html[table_start : table_end]

search_position = 0
new_row = []
data_list = []

# parse table cell by cell
while (search_position >= 0):
    search_position = table_html.find('<td>', search_position)
    if search_position < 0 :
        break;
    search_position += len('<td>')
    
    # get cell
    cell = table_html[search_position : table_html.find('</td>', search_position)]
    
    # remove unnecessary data
    if ('</a>' in cell):
        cell = (cell[cell.find('>') + 1 : cell.find('</a>')])
        
    # add cell data to row
    new_row.append(cell.strip())
    
    # if row is ready to appending (the row has 3 values: Postcode, Borough, Neighbourhood)
    if (len(new_row) == 3) :
        
        # We only process the cells that have an assigned borough. 
        #So we need to ignore cells with a borough that is 'Not assigned'.
        if not new_row[1] == 'Not assigned':
            
            # If neighborhood is 'Not assigned', but borough is not 'Not assigned', then
            # we use the borough name as the neighborhood name
            if new_row[2] == 'Not assigned':
                new_row[2] = new_row[1]
            data_list.append(new_row)
        
        # prepeare to next row from table
        new_row = []
    
        
# create pandas DataFrame
df_raw_table = pd.DataFrame(data_list, columns=['PostalCode', 'Borough', 'Neighborhood'])
df_raw_table.head()

PostalCode           Borough      Neighborhood
0        M3A        North York         Parkwoods
1        M4A        North York  Victoria Village
2        M5A  Downtown Toronto      Harbourfront
3        M5A  Downtown Toronto       Regent Park
4        M6A        North York  Lawrence Heights

### 1.3. Merge (combine) rows with the same postal code
Get list of postcodes

In [26]:
postcodes = df_raw_table.PostalCode.unique()

Copy dataframe to df_merged, replace multiple rows with the same postcode to the single row with the neighborhoods separated with a comma 

In [27]:
df_merged = df_raw_table
for pc in postcodes:
    # get all rows with the certain postocode
    df_pc = df_merged.loc[df_merged['PostalCode'] == pc]
    
    # check: are there multiple rows?
    if len(df_pc.index) > 1 :       
        #print(pc, (df_pc.Neighborhood.to_list()))
            
        # remove these rows
        df_merged = df_merged[df_merged.PostalCode != pc]

        # generate the string as neighborhoods separated with a comma 
        merged_neighbourhood = (', '.join(df_pc.Neighborhood.to_list()))
        
        # append the new row
        df_merged = df_merged.append({
           'PostalCode' : pc , 
           'Borough' : df_raw_table.loc[df_raw_table.PostalCode == pc].iloc[0].Borough, # get Borough by Postcode
           'Neighborhood' : merged_neighbourhood}, 
           ignore_index=True)


print('Rows count: ' + str(df_merged.shape[0]))
df_merged = df_merged.sort_values(by = 'PostalCode').reset_index(drop=True)
#df_merged

Rows count: 103


In [28]:
geo_df = pd.read_csv('http://cocl.us/Geospatial_data')

In [29]:
toronto_data = df_merged.join(geo_df, lsuffix='PostalCode', rsuffix='Postal Code').drop('Postal Code', axis = 1)
toronto_data.head()

PostalCode      Borough                            Neighborhood   Latitude  \
0        M1B  Scarborough                          Rouge, Malvern  43.806686   
1        M1C  Scarborough  Highland Creek, Rouge Hill, Port Union  43.784535   
2        M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3        M1G  Scarborough                                  Woburn  43.770992   
4        M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476

### 1.4. Show map

In [31]:
address = 'Toronto'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        location = [lat, lng],
        radius=5,
        popup=label,
        color= 'blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_newyork)  
    
map_newyork

### 1.5. Define Foursquare Credentials and Version

In [33]:
CLIENT_ID = '1ROEGGL32JNNXXQVZ1XAQM1RN11GXZVG0OPQYTXUES1NZWKJ'
CLIENT_SECRET = 'CV12M1JW2IX22BY2PDG1PPNPFBGRJIWIGOLYX5PFVXF4S1BE' 
VERSION = '20180604'
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

## 2. Explore Neighborhoods
### 2.1 Get nearby venues

In [34]:
def get_nearby_venues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [38]:
toronto_venues = get_nearby_venues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The D

### 2.2. Print amount of the data

In [46]:
print(toronto_venues.shape)
toronto_venues.head()

(2238, 7)


Neighborhood  Neighborhood Latitude  \
0                          Rouge, Malvern              43.806686   
1                          Rouge, Malvern              43.806686   
2  Highland Creek, Rouge Hill, Port Union              43.784535   
3       Guildwood, Morningside, West Hill              43.763573   
4       Guildwood, Morningside, West Hill              43.763573   

   Neighborhood Longitude                            Venue  Venue Latitude  \
0              -79.194353                          Wendy's       43.807448   
1              -79.194353            Interprovincial Group       43.805630   
2              -79.160497            Royal Canadian Legion       43.782533   
3              -79.188711  Swiss Chalet Rotisserie & Grill       43.767697   
4              -79.188711                G & G Electronics       43.765309   

   Venue Longitude        Venue Category  
0       -79.199056  Fast Food Restaurant  
1       -79.200378            Print Shop  
2       -79.163085                   Bar  
3       -79.189914           Pizza Place  
4       -79.191537     Electronics Store

In [78]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
toronto_venues['Venue Category'].unique()

There are 278 uniques categories.


array(['Fast Food Restaurant', 'Print Shop', 'Bar', 'Pizza Place',
       'Electronics Store', 'Spa', 'Mexican Restaurant',
       'Rental Car Location', 'Medical Center', 'Breakfast Spot',
       'Coffee Shop', 'Korean Restaurant', 'Hakka Restaurant',
       'Caribbean Restaurant', 'Thai Restaurant', 'Athletics & Sports',
       'Bank', 'Bakery', 'Fried Chicken Joint', 'Playground',
       'Jewelry Store', 'Department Store', 'Chinese Restaurant',
       'Hobby Shop', 'Train Station', 'Discount Store', 'Intersection',
       'Bus Line', 'Metro Station', 'Bus Station', 'Park', 'Soccer Field',
       'Motel', 'American Restaurant', 'Café', 'General Entertainment',
       'Skating Rink', 'College Stadium', 'Indian Restaurant',
       'Latin American Restaurant', 'Pet Store', 'Vietnamese Restaurant',
       'Light Rail Station', 'Brewery', 'Thrift / Vintage Store',
       'Sandwich Place', 'Middle Eastern Restaurant', 'Shopping Mall',
       'Auto Garage', 'Lounge', 'Italian Restaurant', 

### 2.3. Transform to one-hot vector

In [172]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# there are venue named 'Neighborhood', so we name column as 'Neighborhood_'
# add neighborhood column back to dataframe
toronto_onehot.insert(0, 'Neighborhood_', toronto_venues['Neighborhood'] )


toronto_onehot.head()

Neighborhood_  Accessories Store  Adult Boutique  \
0                          Rouge, Malvern                  0               0   
1                          Rouge, Malvern                  0               0   
2  Highland Creek, Rouge Hill, Port Union                  0               0   
3       Guildwood, Morningside, West Hill                  0               0   
4       Guildwood, Morningside, West Hill                  0               0   

   Afghan Restaurant  Airport  Airport Food Court  Airport Gate  \
0                  0        0                   0             0   
1                  0        0                   0             0   
2                  0        0                   0             0   
3                  0        0                   0             0   
4                  0        0                   0             0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   Antique Shop  Aquarium  Arepa Restaurant  Art Gallery  Art Museum  \
0             0         0                 0            0           0   
1             0         0                 0            0           0   
2             0         0                 0            0           0   
3             0         0                 0            0           0   
4             0         0                 0            0           0   

   Arts & Crafts Store  Asian Restaurant  Athletics & Sports  Auto Garage  \
0                    0                 0                   0            0   
1                    0                 0                   0            0   
2                    0                 0                   0            0   
3                    0                 0                   0            0   
4                    0                 0                   0            0   

   Auto Workshop  BBQ Joint  Baby Store  Bagel Shop  Bakery  Bank  Bar  \
0              0          0           0           0       0     0    0   
1              0          0           0           0       0     0    0   
2              0          0           0           0       0     0    1   
3              0          0           0           0       0     0    0   
4              0          0           0           0       0     0    0   

   Baseball Field  Baseball Stadium  Basketball Court  Basketball Stadium  \
0               0                 0                 0                   0   
1               0                 0                 0                   0   
2               0                 0                 0                   0   
3               0                 0                 0                   0   
4               0                 0                 0                   0   

   Beach  Beer Bar  Beer Store  Belgian Restaurant  Bike Shop  Bistro  \
0      0         0           0                   0          0       0   
1      0         0           0                   0          0       0   
2      0         0           0                   0          0       0   
3      0         0           0                   0          0       0   
4      0         0           0                   0          0       0   

   Boat or Ferry  Bookstore  Boutique  Brazilian Restaurant  Breakfast Spot  \
0              0          0         0                     0               0   
1              0          0         0                     0               0   
2              0          0         0                     0               0   
3              0          0         0                     0               0   
4              0          0         0      

In [173]:
toronto_onehot.shape

(2238, 279)

### 2.4 Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [174]:
toront_grouped = toronto_onehot.groupby('Neighborhood_').mean().reset_index()
toront_grouped

Neighborhood_  Accessories Store  \
0                            Adelaide, King, Richmond           0.000000   
1                                           Agincourt           0.000000   
2   Agincourt North, L'Amoreaux East, Milliken, St...           0.000000   
3   Albion Gardens, Beaumond Heights, Humbergate, ...           0.000000   
4                              Alderwood, Long Branch           0.000000   
5     Bathurst Manor, Downsview North, Wilson Heights           0.000000   
6                                     Bayview Village           0.000000   
7                   Bedford Park, Lawrence Manor East           0.000000   
8                                         Berczy Park           0.000000   
9                         Birch Cliff, Cliffside West           0.000000   
10  Bloordale Gardens, Eringate, Markland Wood, Ol...           0.000000   
11       Brockton, Exhibition Place, Parkdale Village           0.000000   
12  Business Reply Mail Processing Centre 969 Eastern           0.000000   
13                        CFB Toronto, Downsview East           0.000000   
14  CN Tower, Bathurst Quay, Island airport, Harbo...           0.000000   
15                        Cabbagetown, St. James Town           0.000000   
16                                Caledonia-Fairbanks           0.000000   
17              Canada Post Gateway Processing Centre           0.000000   
18                                          Cedarbrae           0.000000   
19                                 Central Bay Street           0.000000   
20          Chinatown, Grange Park, Kensington Market           0.000000   
21                                           Christie           0.000000   
22                               Church and Wellesley           0.000000   
23                    Clairlea, Golden Mile, Oakridge           0.000000   
24            Clarks Corners, Sullivan, Tam O'Shanter           0.000000   
25    Cliffcrest, Cliffside, Scarborough Village West           0.000000   
26  Cloverdale, Islington, Martin Grove, Princess ...           0.000000   
27                     Commerce Court, Victoria Hotel           0.000000   
28                                         Davisville           0.000000   
29                                   Davisville North           0.000000   
30  Deer Park, Forest Hill SE, Rathnelly, South Hi...           0.000000   
31     Del Ray, Keelesdale, Mount Dennis, Silverthorn           0.000000   
32           Design Exchange, Toronto Dominion Centre           0.000000   
33                                    Don Mills North           0.000000   
34  Dorset Park, Scarborough Town Centre, Wexford ...           0.000000   
35                       Dovercourt Village, Dufferin           0.000000   
36                                  Downsview Central           0.000000   
37                                Downsview Northwest           0.000000   
38                                     Downsview West           0.000000   
39        East Birchmount Park, Ionview, Kennedy Park           0.000000   
40                                       East Toronto           0.000000   
41                                   Emery, Humberlea           0.000000   
42                       Fairview, Henry Farm, Oriole           0.016667   
43             First Canadian Place, Underground city           0.000000   
44                   Flemingdon Park, Don Mills South           0.000000   
45                Forest Hill North, Forest Hill West           0.000000   
46                                          Glencairn           0.000000   
47                  Guildwood, Morningside, West Hill           0.000000   
48                     Harbord, University of Toronto           0.000000   
49  Harbourfront East, Toronto Islands, Union Station           0.000000   
50                          Harbourfront, Regent Park           0.000000   
51                      High Park, The Junction South           0.000000   
52          

In [175]:
toront_grouped.shape

(99, 279)

### 2.5. Print each neighborhood along with the top 5 most common venues

In [176]:
num_top_venues = 5

for hood in toront_grouped['Neighborhood_']:
    print("----"+hood+"----")
    temp = toront_grouped[toront_grouped['Neighborhood_'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2           Steakhouse  0.04
3      Thai Restaurant  0.04
4  American Restaurant  0.04


----Agincourt----
            venue  freq
0  Sandwich Place  0.25
1    Skating Rink  0.25
2  Breakfast Spot  0.25
3          Lounge  0.25
4          Museum  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                venue  freq
0          Playground   0.5
1                Park   0.5
2  Miscellaneous Shop   0.0
3       Movie Theater   0.0
4               Motel   0.0


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 venue  freq
0        Grocery Store  0.22
1           Beer Store  0.11
2  Fried Chicken Joint  0.11
3          Coffee Shop  0.11
4       Sandwich Place  0.11


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.22
1             Gym  

                venue  freq
0      Discount Store  0.17
1       Train Station  0.17
2  Chinese Restaurant  0.17
3    Department Store  0.17
4         Coffee Shop  0.17


----East Toronto----
               venue  freq
0  Convenience Store  0.33
1               Park  0.33
2        Coffee Shop  0.33
3  Accessories Store  0.00
4  Mobile Phone Shop  0.00


----Emery, Humberlea----
               venue  freq
0     Baseball Field   1.0
1  Accessories Store   0.0
2             Museum   0.0
3      Movie Theater   0.0
4              Motel   0.0


----Fairview, Henry Farm, Oriole----
                  venue  freq
0        Clothing Store  0.13
1  Fast Food Restaurant  0.08
2           Coffee Shop  0.07
3                Bakery  0.03
4         Women's Store  0.03


----First Canadian Place, Underground city----
                 venue  freq
0                 Café  0.08
1          Coffee Shop  0.08
2                Hotel  0.06
3           Restaurant  0.05
4  American Restaurant  0.04


----Flemingdon

                venue  freq
0         Coffee Shop  0.11
1         Pizza Place  0.08
2                Café  0.08
3    Sushi Restaurant  0.05
4  Italian Restaurant  0.05


----Ryerson, Garden District----
                       venue  freq
0             Clothing Store  0.09
1                Coffee Shop  0.09
2                       Café  0.04
3  Middle Eastern Restaurant  0.03
4             Cosmetics Shop  0.03


----Scarborough Village----
               venue  freq
0         Playground   0.5
1      Jewelry Store   0.5
2  Mobile Phone Shop   0.0
3      Movie Theater   0.0
4              Motel   0.0


----St. James Town----
            venue  freq
0     Coffee Shop  0.07
1      Restaurant  0.06
2           Hotel  0.05
3            Café  0.05
4  Clothing Store  0.04


----Stn A PO Boxes 25 The Esplanade----
          venue  freq
0   Coffee Shop  0.11
1    Restaurant  0.05
2          Café  0.04
3           Pub  0.03
4  Cocktail Bar  0.03


----Studio District----
                 venue  fr

### 2.6. Put that into a pandas dataframe

In [177]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [188]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood_']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood_'] = toront_grouped['Neighborhood_']

for ind in np.arange(toront_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toront_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood_ 1st Most Common Venue  \
0                           Adelaide, King, Richmond           Coffee Shop   
1                                          Agincourt        Sandwich Place   
2  Agincourt North, L'Amoreaux East, Milliken, St...            Playground   
3  Albion Gardens, Beaumond Heights, Humbergate, ...         Grocery Store   
4                             Alderwood, Long Branch           Pizza Place   

  2nd Most Common Venue        3rd Most Common Venue 4th Most Common Venue  \
0                  Café                   Steakhouse       Thai Restaurant   
1                Lounge               Breakfast Spot          Skating Rink   
2                  Park  Eastern European Restaurant        Discount Store   
3           Pizza Place          Fried Chicken Joint           Coffee Shop   
4              Pharmacy                          Pub                   Gym   

  5th Most Common Venue  
0   American Restaurant  
1           Yoga Studio  
2               Dog Run  
3        Sandwich Place  
4        Sandwich Place

## 3. Cluster Neighborhoods

### 3.1. Run k-means to cluster the neighborhood into 5 clusters.

In [189]:
# set number of clusters
kclusters = 10

grouped_clustering = toront_grouped.drop('Neighborhood_', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 7, 3, 7, 0, 0, 0, 0, 0, 0])

### 3.2. Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [190]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [191]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_data
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood_'), on='Neighborhood')

# there are neighborhoods have no venues, so we match them to new cluster (cluster id = count of clusters)  
toronto_merged = toronto_merged.fillna(kclusters)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged.head()

PostalCode      Borough                            Neighborhood   Latitude  \
0        M1B  Scarborough                          Rouge, Malvern  43.806686   
1        M1C  Scarborough  Highland Creek, Rouge Hill, Port Union  43.784535   
2        M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3        M1G  Scarborough                                  Woburn  43.770992   
4        M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0 -79.194353               9  Fast Food Restaurant            Print Shop   
1 -79.160497               5                   Bar           Yoga Studio   
2 -79.188711               0           Pizza Place   Rental Car Location   
3 -79.216917               0           Coffee Shop     Korean Restaurant   
4 -79.239476               0    Athletics & Sports      Hakka Restaurant   

  3rd Most Common Venue        4th Most Common Venue 5th Most Common Venue  
0           Yoga Studio  Eastern European Restaurant        Discount Store  
1      Doner Restaurant                   Donut Shop             Drugstore  
2        Medical Center               Breakfast Spot     Electronics Store  
3           Yoga Studio  Eastern European Restaurant               Dog Run  
4  Caribbean Restaurant              Thai Restaurant                Bakery

### 3.3. Visualize the resulting clusters

In [192]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters